In [1]:
import pandas as pd
from Bio import SeqFeature, Seq, SeqRecord
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import statistics
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_excel("data_set.xlsx", sheet_name=['Sheet0'])
df = pd.concat(df.values(),keys=df.keys(), axis=1)
df.columns = [col[1] for col in df.columns]
df.columns
df.head()

/home/nill/Desktop/Protein-Classification/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Entry,Protein names,Length,DNA binding,Function [CC],Sequence,Protein families,Gene Ontology (molecular function),Gene Ontology IDs
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0003724; GO:0003725; GO:0003968; GO:0004252...
1,A0A087X1C5,Putative cytochrome P450 2D7 (EC 1.14.14.1),515,NaN,FUNCTION: May be responsible for the metabolis...,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,Cytochrome P450 family,aromatase activity [GO:0070330]; heme binding ...,GO:0005506; GO:0005737; GO:0005739; GO:0006805...
2,A0A0B4J2F0,Protein PIGBOS1 (PIGB opposite strand protein 1),54,NaN,FUNCTION: Plays a role in regulation of the un...,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,NaN,NaN,GO:0005741; GO:0006986; GO:1900101
3,A0A0B4J2F2,Putative serine/threonine-protein kinase SIK1B...,783,NaN,FUNCTION: Probable serine/threonine-protein ki...,MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...,"Protein kinase superfamily, CAMK Ser/Thr prote...",ATP binding [GO:0005524]; magnesium ion bindin...,GO:0000287; GO:0004674; GO:0005524; GO:0005634...
4,A0A0B5A886,Envelopment polyprotein (M polyprotein) [Cleav...,1073,NaN,FUNCTION: [Glycoprotein N]: Structural compone...,MMKVIWFSSLICLVIQCSGDSGPIICAGPIHSNKSAGIPHLLGYSE...,Phlebovirus envelope glycoprotein family,NaN,GO:0016020; GO:0039654; GO:0044167; GO:0044178...


In [3]:
df.dropna(subset=['Gene Ontology IDs'], inplace=True)
print(len(df))

25153


In [4]:
df_split = df.assign(Gene_Ontology_IDs = df['Gene Ontology IDs'].str.split('; ')).explode('Gene_Ontology_IDs')

# Drop the original Gene Ontology IDs column
df_split = df_split.drop('Gene Ontology IDs', axis=1)

# Print the modified dataframe
df_split.head()

,Entry,Protein names,Length,DNA binding,Function [CC],Sequence,Protein families,Gene Ontology (molecular function),Gene_Ontology_IDs
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0003724
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0003725
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0003968
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0004252
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0004482


In [5]:
function_cc_counts = df_split['Gene_Ontology_IDs'].value_counts()
total_rows = len(df_split)
list_count =  []
for value, count in function_cc_counts.items():
    percentage = (count / total_rows) * 100
    print(f"\n\nType: {value}, \n\n Count: {count}, \n\n Percentage: {percentage:.2f}%")
    list_count.append(count)
sd = statistics.stdev(list_count)
threshold = sd * 2 




Type: GO:0005634, 

 Count: 5607, 

 Percentage: 1.80%


Type: GO:0005829, 

 Count: 5438, 

 Percentage: 1.74%


Type: GO:0016020, 

 Count: 5421, 

 Percentage: 1.74%


Type: GO:0005886, 

 Count: 5255, 

 Percentage: 1.68%


Type: GO:0005737, 

 Count: 5087, 

 Percentage: 1.63%


Type: GO:0005654, 

 Count: 3869, 

 Percentage: 1.24%


Type: GO:0046872, 

 Count: 3382, 

 Percentage: 1.08%


Type: GO:0003723, 

 Count: 2928, 

 Percentage: 0.94%


Type: GO:0042025, 

 Count: 2322, 

 Percentage: 0.74%


Type: GO:0005576, 

 Count: 2304, 

 Percentage: 0.74%


Type: GO:0070062, 

 Count: 2102, 

 Percentage: 0.67%


Type: GO:0005524, 

 Count: 2017, 

 Percentage: 0.65%


Type: GO:0003677, 

 Count: 1790, 

 Percentage: 0.57%


Type: GO:0042802, 

 Count: 1772, 

 Percentage: 0.57%


Type: GO:0005615, 

 Count: 1699, 

 Percentage: 0.54%


Type: GO:0055036, 

 Count: 1427, 

 Percentage: 0.46%


Type: GO:0006357, 

 Count: 1398, 

 Percentage: 0.45%


Type: GO:0030430, 

 Count: 1

In [6]:
# Assume you have already calculated the standard deviation and threshold

# Create a new DataFrame to store the value counts
value_counts_df = df_split['Gene_Ontology_IDs'].value_counts().reset_index()
value_counts_df.columns = ['Gene_Ontology_IDs', 'Count']

# Filter out the rows where the count is less than the threshold
filtered_df = value_counts_df[value_counts_df['Count'] >= threshold]

# Merge the filtered values back to the original dataframe to drop the rows with count less than the threshold
df_split = pd.merge(df_split, filtered_df, on='Gene_Ontology_IDs', how='inner')

print(len(df_split))
# Print the modified dataframe
df_split.head()

122461


,Entry,Protein names,Length,DNA binding,Function [CC],Sequence,Protein families,Gene Ontology (molecular function),Gene_Ontology_IDs,Count
0,A0A024B7W1,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",GO:0003724,277
1,A0A142I5B9,Genome polyprotein [Cleaved into: Capsid prote...,3423,NaN,FUNCTION: [Capsid protein C]: Plays a role in ...,MKNPKKKSGGFRIVNMLKRGVARVSPFGGLKRLPAGLLLGHGPIRM...,Class I-like SAM-binding methyltransferase sup...,ATP binding [GO:0005524]; ATP hydrolysis activ...,GO:0003724,277
2,A3FMB2,Genome polyprotein [Cleaved into: Capsid prote...,2227,NaN,FUNCTION: [Capsid protein VP1]: Capsid protein...,MNMSKQGIFQTVGSGLDHILSLADIEEEQMIQSVDRTAVTGASYFT...,Picornaviridae polyprotein family,ATP binding [GO:0005524]; cysteine-type endope...,GO:0003724,277
3,A5LGW7,Genome polyprotein [Cleaved into: Capsid prote...,2228,NaN,FUNCTION: [Capsid protein VP1]: Capsid protein...,MNMSRQGIFQTVGSGLDHILSLADVEEEQMIQSVDRTAVTGASYFT...,Picornaviridae polyprotein family,ATP binding [GO:0005524]; cysteine-type endope...,GO:0003724,277
4,B8XTP8,Genome polyprotein [Cleaved into: Capsid prote...,2124,NaN,FUNCTION: [Capsid protein VP1]: Forms an icosa...,MGANNSKESVSSNGNEGTIVNNFYSNQYYASIDASAQGVGTSTTPE...,Picornaviruses polyprotein family,ATP binding [GO:0005524]; ATP hydrolysis activ...,GO:0003724,277


In [7]:
df_split = df_split.sample(n=30, random_state=42)

In [8]:
function_cc_counts = df_split['Gene_Ontology_IDs'].value_counts()
total_rows = len(df_split)
list_count =  []
for value, count in function_cc_counts.items():
    percentage = (count / total_rows) * 100
    print(f"\n\nType: {value}, \n\n Count: {count}, \n\n Percentage: {percentage:.2f}%")



Type: GO:0030430, 

 Count: 2, 

 Percentage: 6.67%


Type: GO:0005829, 

 Count: 2, 

 Percentage: 6.67%


Type: GO:0003677, 

 Count: 2, 

 Percentage: 6.67%


Type: GO:0008270, 

 Count: 2, 

 Percentage: 6.67%


Type: GO:0006508, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0046872, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0005737, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0039526, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0055036, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0000981, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0043025, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0019899, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0005743, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0005739, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0004930, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0005634, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0016020, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0000122, 

 Count: 1, 

 Percentage: 3.33%


Type: GO:0005886, 

 Count

In [12]:
# Initialize an empty list to store dictionaries of subsequences and GO IDs
results_list = []

# Iterate over each row in the DataFrame
for index, row in df_split.iterrows():
    protein_sequence = row['Sequence']
    subseq_len = 20  # Minimum length of subsequence
    
    for i in range(0, len(protein_sequence) - subseq_len + 1):
        subsequence = protein_sequence[i:i+subseq_len]
        go_id = row['Gene_Ontology_IDs']
        
        # Append a dictionary to the results_list
        results_list.append({'Subsequence': subsequence, 'Gene_Ontology_IDs': go_id})

# Create a DataFrame from the results_list
df_results = pd.DataFrame(results_list)

# Print the DataFrame
print(df_results)

                Subsequence Gene_Ontology_IDs
0      METIKSIADMATGVVSSVDS        GO:0006508
1      ETIKSIADMATGVVSSVDST        GO:0006508
2      TIKSIADMATGVVSSVDSTI        GO:0006508
3      IKSIADMATGVVSSVDSTIN        GO:0006508
4      KSIADMATGVVSSVDSTINA        GO:0006508
...                     ...               ...
24022  WQLMGPAGRQDSAPVDPSSP        GO:0006468
24023  QLMGPAGRQDSAPVDPSSPH        GO:0006468
24024  LMGPAGRQDSAPVDPSSPHS        GO:0006468
24025  MGPAGRQDSAPVDPSSPHSY        GO:0006468
24026  GPAGRQDSAPVDPSSPHSYG        GO:0006468

[24027 rows x 2 columns]


In [13]:


X = df_results['Subsequence']
y = df_results['Gene_Ontology_IDs']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with a TfidfVectorizer and a RandomForestClassifier
model = make_pipeline(TfidfVectorizer(analyzer='char', ngram_range=(3, 6)), RandomForestClassifier(n_estimators=100, random_state=42))

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9964627548897211
              precision    recall  f1-score   support

  GO:0000122       1.00      1.00      1.00        90
  GO:0000166       1.00      1.00      1.00        77
  GO:0000287       1.00      1.00      1.00        37
  GO:0000981       1.00      1.00      1.00       193
  GO:0001228       1.00      1.00      1.00       109
  GO:0003677       1.00      1.00      1.00       334
  GO:0003700       1.00      1.00      1.00        48
  GO:0004930       1.00      1.00      1.00        50
  GO:0005634       1.00      1.00      1.00        76
  GO:0005737       1.00      1.00      1.00        31
  GO:0005739       1.00      1.00      1.00        89
  GO:0005743       1.00      1.00      1.00       199
  GO:0005829       1.00      1.00      1.00       732
  GO:0005886       1.00      1.00      1.00        60
  GO:0006468       1.00      1.00      1.00        24
  GO:0006508       1.00      1.00      1.00       398
  GO:0008270       0.98      0.99      0.99       71